In [26]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv('data/events_up_to_01062018.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
user_features = df[['person']].drop_duplicates().copy()
user_features.set_index('person', inplace=True)
user_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


In [29]:
def crear_feature(nombre, evento):
    features = df.loc[df['event'] == evento]
    features = features.groupby('person')['event'].value_counts().unstack()
    features.rename(columns={evento:'cant_'+nombre}, inplace=True)
    return features


In [30]:
conversion = crear_feature('conversions', 'conversion')
user_features = user_features.join(conversion).fillna(0)

In [31]:
conversion = crear_feature('checkouts', 'checkout')
user_features = user_features.join(conversion).fillna(0)

In [32]:
viewed_product = crear_feature('viewed_product', 'viewed product')
user_features = user_features.join(viewed_product).fillna(0)

In [33]:
searched_product = crear_feature('searched_product', 'searched products')
user_features = user_features.join(searched_product).fillna(0)

In [34]:
campaign = crear_feature('ad_campaign_hit', 'ad campaign hit')
user_features = user_features.join(campaign).fillna(0)
user_features["ad_campaign_hit"] = user_features["cant_ad_campaign_hit"].apply(lambda x: True if x>0 else False)
#user_features = user_features.drop("cant_ad_campaign_hit",1)

In [35]:
lead = crear_feature('lead', 'lead')
user_features = user_features.join(lead).fillna(0)
#user_features["lead"] = user_features["cant_lead"].apply(lambda x: True if x>0 else False)
#user_features = user_features.drop("cant_lead",1)

In [36]:
brand_listing = crear_feature('brand_listing', 'brand listing')
user_features = user_features.join(brand_listing).fillna(0)
user_features["brand listing"] = user_features["cant_brand_listing"].apply(lambda x: True if x>0 else False)
#user_features = user_features.drop("cant_brand_listing",1)

In [37]:
user_features.describe().T

,count,mean,std,min,25%,50%,75%,max
cant_conversions,38829.0,0.182621,0.987561,0.0,0.0,0.0,0.0,129.0
cant_checkouts,38829.0,1.682119,2.717728,0.0,1.0,1.0,2.0,197.0
cant_viewed_product,38829.0,32.144119,79.762927,0.0,3.0,10.0,29.0,2355.0
cant_searched_product,38829.0,3.363878,12.897345,0.0,0.0,0.0,2.0,739.0
cant_ad_campaign_hit,38829.0,4.928996,10.821833,0.0,1.0,2.0,5.0,523.0
cant_lead,38829.0,0.025316,0.349768,0.0,0.0,0.0,0.0,38.0
cant_brand_listing,38829.0,5.570888,23.437437,0.0,0.0,0.0,4.0,1891.0


In [38]:
sesiones = df.copy()

In [39]:
sesiones.sort_values(by='timestamp', inplace=True)
sesiones['diff'] = sesiones.groupby('person')['timestamp'].diff() / np.timedelta64(1, 'h')
sesiones['diff'].fillna(0, inplace=True)

In [40]:
sesiones[['timestamp', 'person', 'diff']].head(10)

,timestamp,person,diff
2307205,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753202,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753201,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753200,2018-01-01 08:09:44,0f4e2a4b,0.003611
1753207,2018-01-01 08:45:29,0f4e2a4b,0.595833
2307206,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753210,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753203,2018-01-01 08:45:43,0f4e2a4b,0.003889
1753204,2018-01-01 08:48:57,0f4e2a4b,0.053889
1753213,2018-01-01 08:49:05,0f4e2a4b,0.002222


In [41]:
sesiones['new_session'] = sesiones.groupby('person')['diff'].apply(lambda x: x > 1.0)
sesiones['session_id'] = sesiones.groupby('person')['new_session'].cumsum()

In [42]:
sesiones[['timestamp', 'person', 'event', 'diff', 'session_id']].head()

,timestamp,person,event,diff,session_id
2307205,2018-01-01 08:09:31,0f4e2a4b,visited site,0.000000,0.0
1753202,2018-01-01 08:09:31,0f4e2a4b,search engine hit,0.000000,0.0
1753201,2018-01-01 08:09:31,0f4e2a4b,ad campaign hit,0.000000,0.0
1753200,2018-01-01 08:09:44,0f4e2a4b,viewed product,0.003611,0.0
1753207,2018-01-01 08:45:29,0f4e2a4b,ad campaign hit,0.595833,0.0


In [43]:
cantidad_eventos_usuario = sesiones.groupby(['person', 'session_id'])['event']\
    .value_counts().unstack().unstack().sum(axis=1).to_frame().rename(columns={0:"total_eventos"})
    

In [44]:
cantidad_eventos_usuario.head()

,total_eventos
person,
0008ed71,6.0
00091926,448.0
00091a7a,10.0
000ba417,206.0
000c79fe,17.0


In [45]:
sesiones_por_usuario = sesiones.groupby('person')['session_id'].unique().apply(lambda x: len(x)).\
                        to_frame().rename(columns={'session_id':'total_sesiones'})
promedio_eventos_sesion = cantidad_eventos_usuario.join(sesiones_por_usuario)

In [46]:
promedio_eventos_sesion['promedio_eventos_por_sesion'] = promedio_eventos_sesion['total_eventos']\
                                            / promedio_eventos_sesion['total_sesiones']

In [47]:
user_features = user_features.join(promedio_eventos_sesion[['total_sesiones', 'promedio_eventos_por_sesion']])

In [48]:
user_features.fillna(0, inplace=True)

In [49]:
user_features.to_csv('data/features_basicas.csv', sep=',')

In [50]:
user_features

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,cant_brand_listing,brand listing,total_sesiones,promedio_eventos_por_sesion
person,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,0.0,0.0,False,1,9.000000
ad93850f,0.0,1.0,20.0,0.0,10.0,True,0.0,15.0,True,5,13.000000
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,1.0,4.0,True,89,6.370787
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,0.0,5.0,True,2,13.000000
cccea85e,0.0,1.0,739.0,1.0,15.0,True,0.0,7.0,True,17,49.176471
4c8a8b93,0.0,2.0,177.0,9.0,14.0,True,0.0,8.0,True,14,18.357143
1b9f7cf6,0.0,1.0,9.0,8.0,0.0,False,0.0,3.0,True,3,8.333333
29ebb414,0.0,1.0,8.0,11.0,4.0,True,0.0,0.0,False,4,8.750000
de8fe91b,0.0,1.0,27.0,13.0,0.0,False,0.0,3.0,True,2,26.500000
